pca_w2v_50features 

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_w2v/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_w2v/w2v_withstopwordchangelevels_50_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,0,0.417509,-0.177288,0.900413,5.067122,0.183811,-1.274012,0.757758,0.703022,1.275644,...,-7.347715e-08,2.147511e-07,-9.178981e-09,-6.806621e-08,-2.062242e-07,-2.618753e-07,-7.925330e-09,-4.262707e-08,-1.204524e-07,2.861755e-07
1,1,9.085654,-6.377034,-5.090085,0.604899,2.443397,3.688798,-0.272430,-1.663907,-0.669484,...,9.159759e-08,-1.639009e-07,-1.823723e-07,-2.638463e-07,2.914316e-08,-2.199066e-07,-2.877690e-07,-4.183803e-07,-2.083074e-07,5.104768e-08
2,2,-1.001447,-0.489818,2.204662,5.111655,5.711577,-3.793224,-1.159128,2.779708,-1.511544,...,3.242063e-07,-3.984291e-08,1.858215e-07,2.270497e-07,-6.241087e-07,5.593439e-07,-7.366819e-07,4.260666e-07,-3.139588e-08,-7.329198e-07
3,3,31.469984,13.913559,-1.078216,-0.729067,-2.103110,-7.576188,-1.612798,1.263978,-0.379472,...,8.928165e-08,-2.552782e-07,-7.167630e-08,6.728109e-08,-7.591100e-08,1.595230e-07,6.513076e-08,-1.057089e-07,-2.843075e-07,-5.062684e-08
4,4,3.314504,-2.903750,-1.557225,-7.466617,1.945600,2.302378,-2.478297,1.199961,2.148917,...,-4.376284e-08,3.070699e-07,1.396666e-07,-7.904905e-07,-3.337718e-07,-4.675017e-07,5.909655e-07,7.875010e-08,1.872248e-07,3.729462e-07


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.417509,-0.177288,0.900413,5.067122,0.183811,-1.274012,0.757758,0.703022,1.275644,2.362321,...,-7.347715e-08,2.147511e-07,-9.178981e-09,-6.806621e-08,-2.062242e-07,-2.618753e-07,-7.925330e-09,-4.262707e-08,-1.204524e-07,2.861755e-07
1,9.085654,-6.377034,-5.090085,0.604899,2.443397,3.688798,-0.272430,-1.663907,-0.669484,2.536120,...,9.159759e-08,-1.639009e-07,-1.823723e-07,-2.638463e-07,2.914316e-08,-2.199066e-07,-2.877690e-07,-4.183803e-07,-2.083074e-07,5.104768e-08
2,-1.001447,-0.489818,2.204662,5.111655,5.711577,-3.793224,-1.159128,2.779708,-1.511544,-2.098230,...,3.242063e-07,-3.984291e-08,1.858215e-07,2.270497e-07,-6.241087e-07,5.593439e-07,-7.366819e-07,4.260666e-07,-3.139588e-08,-7.329198e-07
3,31.469984,13.913559,-1.078216,-0.729067,-2.103110,-7.576188,-1.612798,1.263978,-0.379472,-4.522510,...,8.928165e-08,-2.552782e-07,-7.167630e-08,6.728109e-08,-7.591100e-08,1.595230e-07,6.513076e-08,-1.057089e-07,-2.843075e-07,-5.062684e-08
4,3.314504,-2.903750,-1.557225,-7.466617,1.945600,2.302378,-2.478297,1.199961,2.148917,0.728633,...,-4.376284e-08,3.070699e-07,1.396666e-07,-7.904905e-07,-3.337718e-07,-4.675017e-07,5.909655e-07,7.875010e-08,1.872248e-07,3.729462e-07


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 50), (524, 50))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [16]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [17]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3684316939592478


**Decision Tree**

In [18]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [19]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [20]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [21]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [22]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [23]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.481967,0.481967,0.512037,0.482569,0.318702,0.318702,0.348532,0.320913


**Random Forest**

In [24]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [25]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [26]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [27]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    0.9945    0.9972       361
           2     0.9982    1.0000    0.9991       553
           3     0.9967    1.0000    0.9984       306

    accuracy                         0.9984      1220
   macro avg     0.9983    0.9982    0.9982      1220
weighted avg     0.9984    0.9984    0.9984      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2830    0.0955    0.1429       157
           2     0.4655    0.8819    0.6093       237
           3     0.3636    0.0615    0.1053       130

    accuracy                         0.4427       524
   macro avg     0.3707    0.3463    0.2858       524
weighted avg     0.3855    0.4427    0.3445       524



In [28]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.481967,0.481967,0.512037,0.482569,0.318702,0.318702,0.348532,0.320913
1,RandomForest,0.998361,0.998361,0.998365,0.998359,0.442748,0.442748,0.385544,0.344511


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:

classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5632    0.2964    0.3884       361
           2     0.5275    0.8156    0.6406       553
           3     0.5829    0.3333    0.4241       306

    accuracy                         0.5410      1220
   macro avg     0.5578    0.4818    0.4844      1220
weighted avg     0.5519    0.5410    0.5117      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3500    0.1783    0.2363       157
           2     0.4764    0.7679    0.5880       237
           3     0.2419    0.1154    0.1562       130

    accuracy                         0.4294       524
   macro avg     0.3561    0.3539    0.3269       524
weighted avg     0.3804    0.4294    0.3755       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.538525,0.538525,0.555058,0.477073,0.431298,0.431298,0.386934,0.357309
1,RandomForest,0.998361,0.998361,0.998365,0.998359,0.410305,0.410305,0.338871,0.333513
2,Adaboost,0.540984,0.540984,0.551929,0.511682,0.429389,0.429389,0.380378,0.375527


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9892    0.7645    0.8625       361
           2     0.7775    0.9982    0.8741       553
           3     0.9827    0.7418    0.8454       306

    accuracy                         0.8648      1220
   macro avg     0.9165    0.8349    0.8607      1220
weighted avg     0.8916    0.8648    0.8635      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3656    0.2166    0.2720       157
           2     0.4828    0.7722    0.5942       237
           3     0.2308    0.0923    0.1319       130

    accuracy                         0.4370       524
   macro avg     0.3597    0.3603    0.3327       524
weighted avg     0.3852    0.4370    0.3829       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.538525,0.538525,0.555058,0.477073,0.431298,0.431298,0.386934,0.357309
1,RandomForest,0.998361,0.998361,0.998365,0.998359,0.410305,0.410305,0.338871,0.333513
2,Adaboost,0.540984,0.540984,0.551929,0.511682,0.429389,0.429389,0.380378,0.375527
3,GBM,0.864754,0.864754,0.891605,0.863483,0.437023,0.437023,0.385178,0.382942


**SVM Model**

In [29]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [30]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [31]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [32]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3643    0.4127    0.3870       361
           2     0.5249    0.4575    0.4889       553
           3     0.3131    0.3366    0.3244       306

    accuracy                         0.4139      1220
   macro avg     0.4008    0.4023    0.4001      1220
weighted avg     0.4242    0.4139    0.4175      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2961    0.3376    0.3155       157
           2     0.4593    0.4051    0.4305       237
           3     0.2500    0.2615    0.2556       130

    accuracy                         0.3492       524
   macro avg     0.3351    0.3347    0.3339       524
weighted avg     0.3585    0.3492    0.3527       524



In [33]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.481967,0.481967,0.512037,0.482569,0.318702,0.318702,0.348532,0.320913
1,RandomForest,0.998361,0.998361,0.998365,0.998359,0.442748,0.442748,0.385544,0.344511
2,SVM,0.413934,0.413934,0.424246,0.417489,0.349237,0.349237,0.358487,0.352652


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5358    0.4765    0.5044       361
           2     0.5785    0.7794    0.6641       553
           3     0.5000    0.2516    0.3348       306

    accuracy                         0.5574      1220
   macro avg     0.5381    0.5025    0.5011      1220
weighted avg     0.5462    0.5574    0.5342      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3465    0.2803    0.3099       157
           2     0.4681    0.6498    0.5442       237
           3     0.3529    0.1846    0.2424       130

    accuracy                         0.4237       524
   macro avg     0.3892    0.3716    0.3655       524
weighted avg     0.4031    0.4237    0.3991       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.538525,0.538525,0.555058,0.477073,0.431298,0.431298,0.386934,0.357309
1,RandomForest,0.998361,0.998361,0.998365,0.998359,0.410305,0.410305,0.338871,0.333513
2,Adaboost,0.540984,0.540984,0.551929,0.511682,0.429389,0.429389,0.380378,0.375527
3,GBM,0.864754,0.864754,0.891605,0.863483,0.437023,0.437023,0.385178,0.382942
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.557377,0.557377,0.546194,0.534244,0.423664,0.423664,0.403077,0.399105


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3492    0.3047    0.3254       361
           2     0.5033    0.6799    0.5785       553
           3     0.3291    0.1699    0.2241       306

    accuracy                         0.4410      1220
   macro avg     0.3939    0.3849    0.3760      1220
weighted avg     0.4140    0.4410    0.4147      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2764    0.2166    0.2429       157
           2     0.4467    0.6371    0.5252       237
           3     0.2381    0.1154    0.1554       130

    accuracy                         0.3817       524
   macro avg     0.3204    0.3230    0.3078       524
weighted avg     0.3439    0.3817    0.3489       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.538525,0.538525,0.555058,0.477073,0.431298,0.431298,0.386934,0.357309
1,RandomForest,0.998361,0.998361,0.998365,0.998359,0.410305,0.410305,0.338871,0.333513
2,Adaboost,0.540984,0.540984,0.551929,0.511682,0.429389,0.429389,0.380378,0.375527
3,GBM,0.864754,0.864754,0.891605,0.863483,0.437023,0.437023,0.385178,0.382942
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.557377,0.557377,0.546194,0.534244,0.423664,0.423664,0.403077,0.399105
6,GNB,0.440984,0.440984,0.414035,0.414722,0.381679,0.381679,0.343949,0.348879
